# openCypher Workshop - Section 02: Find/Traversal Steps

Now that you have the basics down, let's start working on queries that are more related to traversing a graph structure.  In the following sections. You will use the openCypher patterns necessary for navigating a graph.  This includes crossing common relationships and related nodes along a given traversal path.

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-updated.png"/>



## Following Relationship

Storing relationships in the same manner as entities is a key differentiating factor for graph databases.  Moreso, traversing these relationships is what make graph databases powerful, unique, and fulfills the role of a "purpose-built" tool for querying highly connected datasets.  In the following exercises, we will use openCypher patterns to show how to traverse across relationships in a graph.

### 02.01 - Using ASCII Arrows in MATCH

Let's start with a simple traversal that crosses an relationship.  In the depiction of the data model above, you will notice that Artists are connected to different productions (i.e. movies, tvEpisodes, etc.) via relationships that are labeled with the different credits or roles.  If we wanted to find all productions that a given Artist worked on, we can traverse across a relationship with a given credit/role to the productions related to that Artist.  Let's give this a try.  

In the following example, we will find the movies directed by Christopher Nolan. In this example, we start by finding the nodes representative of Christopher Nolan and then we use the arrow notation in the `MATCH` clause to traverse across the incoming relationship with the relationship type of _director_.  At this point, we now have a list of all of the nodes related to productions of which Christopher Nolan directed.

__NOTE__: Relationships in Neptune have directionality.  You can supply a `MATCH` pattern without a direction if you want to ignore the direction of an edge.  

In [ ]:
%%oc

MATCH (a:Artist)<-[:director]-(m:movie)
WHERE a.name = 'Christopher Nolan'
RETURN m.title

### Using `labels()` and `type()`

As mentioned above, you can also filter on labels using the `labels()` or `type()` functions.  The same query above can be expressed using the following:

In [ ]:
%%oc

MATCH (a)<-[e]-(m)
WHERE a.name = 'Christopher Nolan' 
                AND 'Artist' IN labels(a)
                AND type(e) = 'director'
                AND 'movie' IN labels(m)
RETURN m.title

### 02.02 - Counting Things

Similarly to the example above, we can traverse across outgoing relationships as well.  In this next example, we are using the `Person` and `Artist` nodes along with the `follows` relationships.  Run the following code block to find the person with ID 'person378' and count the number of people they follow.

We also introduce a new function in this example, `count()`.  Count does exactly as it implies.  It takes whatever input is mapped to it and outputs a count of those objects.

In [ ]:
%%oc

MATCH (n)-[:follows]->(a)
WHERE id(n) = 'person378'
RETURN count(a)

### 02.03 - Multiple relationships

Here's an example where you can chain node and relationship patterns together to navigate deeper into a graph. In the following example, we find the Person with ID 'person378' and count the number of unique people who follow the same artists.

You will notice that we're also using another new term here, `DISTINCT`.  This retrieves only unique values from the output of the `RETURN` clause.  We need `DISTINCT` in this case as there may be patterns where we see the same people following different artists.  We don't care about the duplicates here, as we only want a count of the unique `Person` nodes that follow the same artists as `Person` node 'person378'.

In [ ]:
%%oc

MATCH (p1)-[:follows]->(a)<-[:follows]-(p2)
WHERE id(p1) = 'person378'
RETURN count(DISTINCT p2)

### 02.04 - Creating custom results headers with `AS`

Sometimes it is beneficial to label the different columns in the output with specific names.  This is possible via aliasing with `AS`.
 
As an example, say we wanted to output everyone that co-starred with Halle Berry and the movies in which they appeared together. We want to provide the results in a table of movie title and co-star name.  Let's try that via the following query:

In [ ]:
%%oc

MATCH (a:Artist)<-[:actress|actor]-(m:movie)-[:actress|actor]->(c:Artist)
WHERE a.name = 'Halle Berry'
RETURN m.title AS movie, c.name AS costar

Note how this last query did NOT include Halle Berry in the resulting list of co-stars.  openCypher has a computational constraint where each edge can be traversed only once.  As such, the traversal will not cycle back on itself.  This is different behavior than Gremlin, where one must consider cycles as part of the traversal.

### 02.05 - Chaining within queries using `WITH`

When writing queries, you might find yourself in a situation where you need to make some computions before continuing on in the query. With the `WITH` clause, you can output the results of one sub-query to be used in subsequent sub-queries. This can be useful for situations like needing to sort results first, or needing to make a collection of results first. 

One common way that `WITH` is used is to limit the number of results that are passed on to future `MATCH` statements. For example, if we want to get the co-stars of Tom Cruise's top 2 highest ratest movies, we can do the following:

In [ ]:
%%oc

MATCH (a:Artist {name: 'Tom Cruise'})<-[:actor|:actress]-(m:movie)
WHERE exists(m.averageRating)
WITH id(m) AS movie, m.averageRating AS rating, a AS tom
ORDER BY rating DESC
LIMIT 2
MATCH (x:movie)-[:actor|:actress]->(y:Artist)
WHERE id(x) = movie AND y <> tom
RETURN distinct y.name, x.title

First, we use a match pattern to find all the movies that Tom Cruise has been in. We'll also use the `WHERE` clause with the predicate function `exists` to limit patterns to movies that have a rating.

In [ ]:
%%oc

MATCH (a:Artist {name: 'Tom Cruise'})<-[:actor|:actress]-(m:movie)
WHERE exists(m.averageRating)
RETURN m

Now we'll have to sort all of the movies by rating, then grab the top 2 highest rated movies. We use `WITH` to pull the output from the first `MATCH` so we can use that information to sort the movies by rating, and to also pass that information to the next `MATCH`/`WHERE` clauses.

In [ ]:
%%oc

MATCH (a:Artist {name: 'Tom Cruise'})<-[:actor|:actress]-(m:movie)
WHERE exists(m.averageRating)
WITH id(m) AS movie, m.averageRating AS rating, a AS tom
ORDER BY rating DESC
LIMIT 2
RETURN movie

We used `WITH` to pass on the IDs of the top 2 highest rated movies, so that in our second `MATCH`, we can limit patterns to only those where the movie is one of the top 2 highest rated movies. 

In [ ]:
%%oc

MATCH (a:Artist {name: 'Tom Cruise'})<-[:actor|:actress]-(m:movie)
WHERE exists(m.averageRating)
WITH id(m) AS movie, m.averageRating AS rating, a AS tom
ORDER BY rating DESC
LIMIT 2
MATCH (x:movie)-[:actor|:actress]->(y:Artist)
WHERE id(x) = movie AND y <> tom
RETURN distinct y.name, x.title

### 02.06 - Using `COLLECT()` to create aggregations and `UNWIND` to unroll them

Let's trying something a bit more complex.  Ever wondered how Facebook, Twitter, or LinkedIn provide you with a list of people that you "might want to connect with"?  This type of traversal can also be referred to as the __Friends of Friends__ (FoF) traversal. 

We can simulate FoF in our IMDb dataset by looking at co-stars of co-stars for a particular performer.  We can basically ask "what performers should our unique performer look to star with based on their total number of common co-stars".

In doing this type of query, we'll need to leverage the `COLLECT()` clause.  The `COLLECT()` clause creates a list containing the values fed into it. We'll need this function to create a temporary collection within the middle of our traversal that we can refer back to.  In this case, we'll want to aggregate all co-stars of a given performer, then the co-stars of co-stars, and then count the number of times the co-stars of co-stars appear across common co-star connections.  We will also want to filter out co-stars and our given performer from the results.

See the following documentation:
- [`COLLECT()`](https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf), page 152
- [`UNWIND`](https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf), page 81

In the following example, we'll use Rebel Wilson and find which performer she has not appeared with but shares the highest number of common co-stars.


First, we use `MATCH()` to find the co-stars of Rebel, which are given by `(a2)`. 

In [ ]:
%%oc

MATCH (rebel:Artist {name: 'Rebel Wilson'})<-[:actor|actress]-(m1)-[:actor|actress]->(a2) 
RETURN a2

There's a possibility that co-stars of co-stars could actually be direct co-stars with Rebel through a different connection. Since we only want to recommend people that Rebel has never acted with before (i.e. people that are not direct co-stars), we'll need a way to check if there's any overlap between the collection of co-stars and the collection of co-stars of co-stars. We can do this using `COLLECT()` to gather our co-stars as a list, which we'll use later on to make sure our co-stars of co-stars aren't in that list. We use `WITH` to pipe our collection to the next part of the query.

In [ ]:
%%oc

MATCH (rebel:Artist {name: 'Rebel Wilson'})<-[:actor|actress]-(m1)-[:actor|actress]->(a2) 
WITH rebel, COLLECT(a2) AS costars // pass the node representing Rebel to the next part of the query so we can 
                                   // remove her from the co-stars of co-stars that we find
                                   // also collect co-stars found as a list called 'costars'
RETURN costars

Before we do our second `MATCH` clause, we need to `UNWIND` our collection of co-stars back to individual co-stars so we can reference each co-star occurence as part of the matching pattern. We'll also use `WHERE` to filter out co-stars of co-stars that are also present in our collection of co-stars, and remove Rebel from the co-stars of co-stars since she can't be her own co-star of co-star.

In [ ]:
%%oc

MATCH (rebel:Artist {name: 'Rebel Wilson'})<-[:actor|actress]-(m1)-[:actor|actress]->(a2) 
WITH rebel, COLLECT(a2) AS costars 
UNWIND costars AS a2 
MATCH (a2)<-[:actor|actress]-()-[:actor|actress]->(a3) 
WHERE NOT(a3 IN costars) // only keep matches where the co-star of co-star is not in the list of co-stars
      AND rebel <> a3 // only keep matches where Rebel is not a co-star of co-star
RETURN a3 // return the resulting co-stars of co-stars

Now we have a list of all the co-stars of co-stars of Rebel that she has never acted with (i.e. not her direct co-star through a different connection). Some people show up more than once, so we'll group them by name and count the number of occurences of each. We then order people by how many occurences they have - similar to how a social platform gives you friend suggestions based on how many mutual connections you have with that person.

In [ ]:
%%oc

MATCH (rebel:Artist {name: 'Rebel Wilson'})<-[:actor|actress]-(m1)-[:actor|actress]->(a2) 
WITH rebel, COLLECT(a2) AS costars 
UNWIND costars AS a2 
MATCH (a2)<-[:actor|actress]-()-[:actor|actress]->(a3) 
WHERE NOT(a3 IN costars) AND rebel <> a3 
RETURN a3.name, count(*) AS score 
ORDER BY score DESC

### Review

In this notebook we started using openCypher to traverse across both vertices and edges in a graph.  We displayed how these queries can be used to take advantage of the first class representation of relationships in a graph.  Lastly, we also looked at different types of aggregations and how to display results in an ordered fashion.  We will continue building on the use of these patterns in the next notebook.

### End of Section 02 - Move onto the next notebook.